In [1]:
import pandas as pd
import numpy as np


from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from scipy.sparse import csc_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('data.csv')
del data['order_ts']

In [3]:
learning_data = data.sample(frac=0.70, random_state=42)

In [4]:
test_data = data.drop(learning_data.index).sample(frac=0.15, random_state=42)

Весь датасет разбит на обучающую и тестовую выборку. В обучающей выборке находится 75% всех данных, в тестовой - 25% данных.

In [6]:
test_data

,user_id,item_id
9894930,985594,3356
5289373,850799,233
14153829,69203,3373
11900177,782799,180
8012515,36074,212
...,...,...
7369352,928228,11
11355654,879490,352
3555318,765580,3416
14253140,187714,96


In [7]:
len(learning_data) + len(test_data)

15843009

Модель будет предсказывать id товара для некоторого пользователя. Основной гипотезой будет то что, товары схожи, если их покупали одни и те же пользователи. Каждому товару в задаче будут сопоставлены все пользователи, купившие его. Таким образом, каждому пользователю будут предложены товары тех пользователей, которые покупали схожие с ним товары. В задаче возможен как cold start, так и warm start, так как выборка разбита случайно и возможно появление пользователей без истории покупок. Будем использовать implicit kNN и матричную факторизацию.

В качестве метрик оценки качества, будем использовать precision и recall, а также MAP@K. Эта метрика хороша тем, что эта метрика позволяет получить обобщенную оценку качества, настраиваема и устойчива к невбалансированным данным.

In [10]:
users = learning_data['user_id'].value_counts()

In [11]:
learning_data

,user_id,item_id
17408196,478886,2592
2445233,984912,744
5529204,691576,162
16900811,153459,593
233826,937639,943
...,...,...
13298027,199141,305
12853806,598910,2833
7528213,140336,1611
9924580,690375,406


In [12]:
learning_data['count'] = 1
learning_data

,user_id,item_id,count
17408196,478886,2592,1
2445233,984912,744,1
5529204,691576,162,1
16900811,153459,593,1
233826,937639,943,1
...,...,...,...
13298027,199141,305,1
12853806,598910,2833,1
7528213,140336,1611,1
9924580,690375,406,1


In [13]:
# sorted_data = learning_data.sort_values(by='item_id')
# sorted_data
# interaction_matrix = [[0 for i in range(len(learning_data['item_id']))] for j in range(len(learning_data['user_id']))]
# for user_id, item_id in interactions:
#     user_index = learning_data.index(user_id)
#     item_index = learning_data.index(item_id)
#     interaction_matrix[user_index][item_index] = 1 
# interaction_matrix = learning_data.pivot_table(index='user_id', columns='item_id', fill_value=0)

interaction_matrix = csc_matrix((learning_data['count'], (learning_data['user_id'], learning_data['item_id'])))

In [14]:
model = NearestNeighbors(n_neighbors=1, metric='cosine')
model.fit(interaction_matrix)

NearestNeighbors(metric='cosine', n_neighbors=1)

In [15]:
print(interaction_matrix[478886, 359])

3


In [21]:
recs = []
count = 0
for item_id in test_data['item_id'].head(10000):
    distances, indices = model.kneighbors(interaction_matrix[item_id], n_neighbors=1)
    recommended_items = indices.flatten()
    recs.append(recommended_items)
    

In [22]:
def average_precision(recommended, actual, k):
    recommended = recommended[:k]  # Ограничиваем до K
    relevant_items = 0
    precision_sum = 0.0

    for i, item in enumerate(recommended):
        if item == actual:
            relevant_items += 1
            precision_sum += relevant_items / (i + 1)

    if relevant_items == 0:
        return 0.0  # Если нет релевантных товаров, AP = 0

    return precision_sum / relevant_items  # AP

def mean_average_precision(recommendations, actuals, k):
    ap_list = []
    for recommended, actual in zip(recommendations, actuals):
        ap = average_precision(recommended, actual, k)
        ap_list.append(ap)
    return sum(ap_list) / len(ap_list) if ap_list else 0.0  # MAP

# mean_average_precision(recs, learning_data['item_id'].values, 1)

In [23]:
mean_average_precision(recs, test_data['item_id'].values, 1)

0.9239